<a href="https://colab.research.google.com/github/brownnyi/Project_FakeNews-Detector/blob/main/team3_fakenews_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Koalpaca를 사용하는데 필요한 라이브러리 설치

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00


#Koalpaca를 파인튜닝하기 위해 데이터셋 구성

In [ ]:
from datasets import load_dataset

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
data = [
{"instruction":"제니와 뷔가 결별했다는 기사가 진짜야?",
"input":"",
"output":'''
사실입니다. 수 차례 열애설이 불거졌던 그룹 '방탄소년단'의 멤버 뷔 씨와 걸그룹 '블랙핑크'의 멤버 제니 씨가 결별했다는 보도가 나왔습니다. 지난 6일 한 매체는 뷔 씨와 제니 씨가 최근 연인 관계를 정리했다고 보도했는데요. 이와 관련해 두 사람의 소속사인 빅히트뮤직과 YG엔터테인먼트는 앞서 열애설이 불거졌을 때와 마찬가지로 어떠한 입장도 내놓지 않았습니다.뷔 씨와 제니 씨의 열애설은 지난해 5월 처음 불거졌습니다. 뷔 씨와 제니 씨로 추정되는 남녀가 제주도에서 드라이브를 했다는 목격담과 함께 사진이 퍼진 건데요.1년 만인 지난 5월에도 뷔 씨와 제니 씨로 추정되는 남녀가 프랑스 파리 센강 인근에서 손을 잡고 밤 산책을 즐기는 모습이 포착되면서 열애설에 불을 지폈습니다.
'''},
{"instruction":"뷔가 군대에 간다는 기사가 진짜야?",
"input":"",
"output":'''
사실입니다. 방탄소년단(BTS)의 RM과 뷔가 오늘(11일) 현역으로 육군에 입대한다.내일(12일)은 지민과 정국까지 입대한다. 앞서 입대한 진과 제이홉, 슈가까지 방탄소년단 전원이 병역 의무 이행에 들어가는 셈이다. RM은 "지난 10년간 방탄소년단으로 살 수 있어서 너무 행복했다"며 "이 시기가 우리 모두에게 낯설고 새로운 영감과 배움의 시기가 될 거라 믿는다"는 인사를 남겼다. RM은 "잠시 동안 안녕"이라며 "언제 어디서든 우리가 우리이도록! 미래에서 만나자"고 했다. 뷔는 "같이 (군대에) 가는 친구들이 저보다 어릴 수 있겠지만 그들과 이야기하면서 새로움도 얻을 수 있을 것"이라며 "설렌다"고 말했다. RM, 뷔, 지민, 정국의 전역 예정일은 오는 2025년 6월이다. 방탄소년단 멤버들과 소속사는 2025년 완전체 활동 재개를 희망하고 있다.
'''},
{"instruction":"지수와 뷔가 연애했다는 뉴스가 사실이야?",
"input":"",
"output":'''
거짓입니다. 뷔는 지수와 같은 그룹의 멤버인 제니와 연애를 하다 지난 12월 6일 결별 사실을 알렸습니다. 블랙핑크 지수(28)와 배우 안보현(35)이 결별, 연인에서 다시 동료로 돌아갔다. 한 연예 관계자는 24일 JTBC엔터뉴스에 "지수, 안보현이 바쁜 스케줄로 자연스럽게 관계가 소원해져 최근 결별했다"라고 전했다. 이와 관련 지수의 소속사 YG엔터테인먼트 측은 "결별이 맞다"라고 짤막한 입장을 밝혔고, 안보현의 소속사 FN엔터테인먼트 역시 같은 입장으로 결별 사실을 인정했다. 지난 8월 지수, 안보현은 열애를 공식화했다. 두 사람이 데이트 중인 모습이 한 매체를 통해 포착됐다. 당시 지수, 안보현의 소속사 측은 "좋은 감정으로 서로를 조금씩 알아가고 있는 단계다. 두 사람을 따뜻한 시선으로 지켜봐 달라"라며 열애를 인정했다. 특히 블랙핑크 내 첫 열애 인정이라 더욱 화제를 모았다. 7살 나이 차를 뛰어넘어 연예계 공개 커플 대열에 합류했던 지수, 안보현은 아쉽게도 공개 열애 두 달 만에 결별, 각자의 일에 보다 집중하기로 했다.
'''},
{"instruction":"뉴진스가 이낙연을 응원한다는 기사는 진짜야?",
"input":"",
"output":'''
거짓입니다. 이낙연 전 국무총리가 더불어민주당을 벗어나 신당 창당을 선언한 상황에서 뉴진스와 블랙핑크, 방탄소년단(BTS) 등 유명 인사들이 등장하는 이낙연 신당 ‘응원 포스터’가 논란이다. 친낙계(친이낙연계)는 해당 포스터에 대해 “모임과 무관한 가짜”라고 밝혔다. 18일 소셜미디어(SNS) 등에서는 ‘이낙연 신당’을 응원하는 포스터 이미지가 확산하고 있다. 포스터를 보면 ‘이낙연 신당 2024. 01’ ‘양당체제 이낙연이 바꿉니다’ ‘이낙연 신당 국민이 원합니다’ 등 문구가 적혀있다. 문구 밑에는 이 전 총리와 함께 정치인, 탤런트, 가수 등 유명인들이 다수 등장한다. 왼쪽 위부터 시계 방향으로 문재인 전 대통령, 가수 아이유, 보이그룹 BTS, 걸그룹 뉴진스와 블랙핑크, 피겨선수 김연아, 트로트 가수 임영웅, 축구선수 손흥민, 방송인 유재석 등의 사진이 들어갔다. 포스터 중앙에는 이 전 대표가 위치했다. 이들 사진과 함께 ‘네 국민이 원하면 만들겠습니다’ ‘새로운 민주당 이재명 없는 민주당’ 문구와 함께 ‘민주주의실천행동’이라고 적혀있다. 민주주의실천행동은 친낙계 인사들이 주로 모인 원외모임이다. 민주주의실천행동은 이에 대해 “해당 웹자보는 이재명 지지자로 추정되는 X(옛 트위터) 사용자의 허위조작정보 유포로 이 같은 웹자보를 발행한 사실이 없다”고 말했다. 이어 “해당 사용자는 그동안 반복해서 문재인 전 대통령 지지자, 혹은 이낙연 전 총리의 지지를 사칭해 허위조작정보를 생산해 왔다”며 “이 대표를 지지하는 정치권 원외인사들과 유튜버들을 중심으로 확산하는 패턴이 반복되고 있다”고 설명했다. 반대 진영 지지자가 의도적으로 ‘과한 응원 포스터’를 제작해 유포함으로써 이 전 총리에 대한 반감을 유도하려 했다는 주장이다. 이에 대해 포스터를 제작한 것으로 추정되는 인물은 별다른 입장을 내놓지 않았다.
'''},
{"instruction":"지드래곤(권지용)이 마약 음성이 나왔다는 뉴스가 사실이야?",
"input":"",
"output":'''
사실입니다. 마약 투약 혐의를 받았으나 정밀 검사에서 음성 판정을 받은 가수 지드래곤(35·본명 권지용)씨가 혐의 없음으로 불송치 될 예정이다. 13일 경찰 등에 따르면 인천경찰청 마약범죄수사대는 마약 투약 혐의로 수사 중인 권씨를 혐의 없음으로 불송치할 예정이다. 권씨가 정밀 검사에서 음성 판정을 받았고, 최근 참고인 6명을 조사했지만 혐의를 입증할 만한 진술을 확보하지 못해서다. 경찰은 또 지난달 25일쯤 만료된 권씨의 출국금지 기간을 연장하지 않기로 했다. 경찰 관계자는 “권 씨에 대해선 혐의 없음으로 사건을 마무리하려고 한다”며 “12월 말쯤 (혐의없음으로)최종 결론을 낼 예정”이라고 말했다. 앞서 경찰은 지난 9월 서울 강남의 한 유흥업소 실장 A씨(29·여)의 진술을 토대로 권지용이 지난해 12월 해당 유흥주점에서 마약을 투약한 것은 아닌지 의심하고 수사에 나섰다. 지드래곤은 지난달 25일 마약류 관리에 관한 법률 위반 혐의로 불구속 입건됐으나, 혐의를 전면 부인한 바 있다. 이후 지난달 6일 경찰에 자진 출석한 권씨는 “마약 투약 사실이 없다”라고 혐의를 전면 부인했다. 그 결과 간이 시약 검사와 모발 감정에 이어 국립과학수사연구원의 손·발톱 정밀감정에서도 마약 음성 판정을 받았다. 이를 두고 경찰이 무리한 수사를 한 것 아니냐는 비판이 일었다. 하지만 경찰은 진술을 토대로 수사를 하는 것이 원칙이고, 혐의가 없음을 밝혀내는 것도 경찰의 역할이라고 밝혔다.
'''},
{"instruction":"배우 현빈과 손예진이 이혼한다는 기사가 진짜야?",
"input":"",
"output":'''
거짓입니다. 배우 현빈·손예진 부부가 가짜 뉴스의 피해자가 됐다. 지난 3월 15일 한 유튜브 채널에는 '현빈 손예진 결혼 6개월 만에 이혼..충격이네요!'라는 제목의 영상이 게재돼 눈길을 끌었다. 영상엔 현빈이 필리핀 마닐라에서 도박빚으로 150억 원을 날렸고, 이를 봐주고 봐준 손예진이 결국 이혼을 선언했다는 얼토당토 않는 주장이 담겼다. 또 현빈의 도박빚으로 손예진이 가지고 있던 강남 빌딩 5채가 날아갔고, 현빈을 마닐라에서 봤다는 목격담까지 담겼다. 연예계에서 초고속으로 이혼한 부부로 이름이 남게 됐다는 주장으로 영상은 마무리 됐다. 이 유튜브 채널은 지난 3월 14일에도 현빈·손예진 부부의 이혼을 언급했다. 영상에서 작성자는 현빈과 손예진이 오는 4월 이혼을 앞두고 있으며, 현빈이 손예진 몰래 주식 투자를 하다 30억 원의 손실을 봤다고 주장했다. ABC 토크쇼 '지미 키멜 라이브'의 화면 일부를 붙이며 미국 생방송에까지 나온 내용이라고 해 황당함을 자아냈다. 현빈, 손예진 소속사 측은 가짜 뉴스에 발끈하며 "적법한 조치를 취할 예정"이라고 입장을 밝혔다. 현빈·손예진, 지난해 3월 결혼→11월 득남
현빈과 손예진은 지난해 3월 결혼식을 올리며 '세기의 부부' 타이틀을 얻었다. 그리고 손예진이 지난해 11월 27일 아들을 낳으며 두 사람은 부모가 됐다. 엄마가 된 손예진은 육아로 바쁜 일상을 보내고 있으며, 현빈은 영화 '하얼빈' 촬영 도중 아내와 아들을 살뜰히 챙겼다. 현빈·손예진 부부에 앞서 선우은숙·유영재 부부, 최수종·하희라 부부, 백종원·소유진 부부, 김연아·고우림 부부 등도 가짜 뉴스의 피해를 봤다. 이들 부부들은 가짜 뉴스에 강력한 대응을 예고했지만, 여전히 영상은 이어지고 있으며 이를 믿는 대중들도 많아 적지 않은 피해가 예상된다. 가짜 뉴스의 수위는 살아있는 사람도 죽은 사람으로 둔갑시키며 눈살을 찌푸리게 한다. 지난해 유방암 투병 중인 서정희는 자신의 영정사진까지 만들어 썸네일에 노출한 가짜 뉴스에 분노를 쏟아내기도 했다.
'''},
{"instruction":"두 기사를 읽고 내용의 유사도를 평가해줘",
"input":'''
첫번째 기사 - 배우 현빈과 손예진이 이혼한다. 손예진 임신 중 현빈의 간통 증거. 이것은 손예진에게 많은 고통과 상처를 야기했지만 그녀는 여전히 침착하게 기다리며 진정한 사랑으로 돌아올 남편을 기다리고 있었습니다. 그러나 그녀는 현빈과의 연인 관계를 끝내고 새로운 삶으로 나아가기로 결정했습니다. 현빈의 외도는 그를 지지하는 팬들로부터 많은 비판을 받게 만들었습니다. 그러나 이는 엔터테인먼트 산업 내 다른 인물들에게 경고가 되었으며 그들도 인간이면 잘못된 행동은 후회하게 될 것임을 보여줍니다. 두번째 기사 - 배우 손예진이 임신 중인 사실을 인정했다. 손예진 측은 이에 대해 긍정적인 반응을 보이며 현빈과의 가정을 이어가기로 했다고 밝혔다. 손예진은 현재 건강한 임신 생활을 유지하며 촬영 현장과 가정을 오가며 바쁜 일정을 소화하고 있다. 현빈 또한 손예진의 임신 소식에 기뻐하며 가족을 위해 최선을 다하겠다고 전했다. 두 주인공은 이번 사건을 통해 더욱 깊어진 가족의 유대감을 나타내고 있다.
''',
"output":'''
주어진 두 기사의 내용을 바탕으로 유사도를 평가하겠습니다. 두 기사는 주제가 유사하면서도 서로 다른 내용을 다루고 있습니다. 첫 번째 기사에서는 배우 현빈과 손예진의 이혼과 간통 증거에 대한 내용이 주를 이루고 있으며, 손예진이 새로운 삶으로 나아가기로 결정했다는 내용도 포함되어 있습니다. 반면 두 번째 기사에서는 손예진의 임신과 이에 대한 긍정적인 반응, 가정을 이어가기로 한다는 내용이 주를 이루고 있습니다. 두 기사 간의 공통점은 손예진이라는 인물에 중점을 두고 있으며, 양측에서 손예진의 결정에 대한 긍정적인 응원이 언급되고 있습니다. 그러나 첫 번째 기사에서는 부정적인 측면과 간통 증거에 대한 강조가 있고, 두 번째 기사에서는 긍정적인 측면과 임신 소식에 대한 강조가 있습니다. 두 기사 간의 유사도는 전반적인 주제와 몇몇 공통된 단어나 구절을 제외하면 상대적으로 낮다고 평가할 수 있습니다.
'''},
{"instruction":"두 기사를 읽고 내용의 유사도를 평가해줘. 첫번째 기사 - 배우 현빈과 손예진이 이혼한다. 손예진 임신 중 현빈의 간통 증거. 이것은 손예진에게 많은 고통과 상처를 야기했지만 그녀는 여전히 침착하게 기다리며 진정한 사랑으로 돌아올 남편을 기다리고 있었습니다. 그러나 그녀는 현빈과의 연인 관계를 끝내고 새로운 삶으로 나아가기로 결정했습니다. 현빈의 외도는 그를 지지하는 팬들로부터 많은 비판을 받게 만들었습니다. 그러나 이는 엔터테인먼트 산업 내 다른 인물들에게 경고가 되었으며 그들도 인간이면 잘못된 행동은 후회하게 될 것임을 보여줍니다.두번째 기사 - 배우 손예진이 임신 중인 사실을 인정했다. 손예진 측은 이에 대해 긍정적인 반응을 보이며 현빈과의 가정을 이어가기로 했다고 밝혔다. 손예진은 현재 건강한 임신 생활을 유지하며 촬영 현장과 가정을 오가며 바쁜 일정을 소화하고 있다. 현빈 또한 손예진의 임신 소식에 기뻐하며 가족을 위해 최선을 다하겠다고 전했다. 두 주인공은 이번 사건을 통해 더욱 깊어진 가족의 유대감을 나타내고 있다.",
"input":"",
"output":'''
주어진 두 기사의 내용을 바탕으로 유사도를 평가하겠습니다. 두 기사는 주제가 유사하면서도 서로 다른 내용을 다루고 있습니다. 첫 번째 기사에서는 배우 현빈과 손예진의 이혼과 간통 증거에 대한 내용이 주를 이루고 있으며, 손예진이 새로운 삶으로 나아가기로 결정했다는 내용도 포함되어 있습니다. 반면 두 번째 기사에서는 손예진의 임신과 이에 대한 긍정적인 반응, 가정을 이어가기로 한다는 내용이 주를 이루고 있습니다. 두 기사 간의 공통점은 손예진이라는 인물에 중점을 두고 있으며, 양측에서 손예진의 결정에 대한 긍정적인 응원이 언급되고 있습니다. 그러나 첫 번째 기사에서는 부정적인 측면과 간통 증거에 대한 강조가 있고, 두 번째 기사에서는 긍정적인 측면과 임신 소식에 대한 강조가 있습니다. 두 기사 간의 유사도는 전반적인 주제와 몇몇 공통된 단어나 구절을 제외하면 상대적으로 낮다고 평가할 수 있습니다.
'''},
{"instruction":"손예진과 현빈이 이혼했다는 뉴스가 사실이야?",
"input":"",
"output":'''
거짓입니다. 배우 현빈과 손예진 부부가 이혼설 가짜뉴스에 피해를 받고 있다. 지난 15일 유튜브에는 '현빈 손예진 결혼 6개월 만에 이혼... 충격이네요!!' 라는 제목의 영상이 게재됐다. 이 영상에는 현빈이 해외에서 도박을 했으며, 이로 인해 손예진이 재산상 손실을 입어 합의 이혼을 했다는 다소 자극적이고 충격적인 내용이 담겼다. 그러나 이 영상은 유튜브발 가짜뉴스다. 현빈과 손예진 부부는 지난해 3월 말에 결혼했다. 영상 내용의 주장처럼 6개월 만에 이혼했다면 지난해 9월이나 10월 시기적으로 맞으나 해당 영상에서는 올해 3월 15일에 이혼했다고 했다. 부부는 2018년 영화 협상과 드라마 사랑의 불시착에서 호흡을 맞추면서 인연을 맺었고 1년 간 공개 열애 후 지난해 3월 결혼식을 올렸다. 이후 같은 해 6월 임식 소식을 알린 후 11월 득남했다.
'''},
{"instruction":'''
두 기사를 읽고 내용의 유사도를 평가해줘. 첫번째 뉴스 - 피겨 여왕 김연아의 남편, 고우림이 결혼 1년 만에 군 복무에 입대했다. 군 복무 이전의 고우림은 팀 포레스텔라의 멤버로 활동하며 뛰어난 실력과 훌륭한 인성으로 팬들의 사랑을 받았다. 결혼 후 1년간 행복한 신혼 생활을 즐겼지만, 최근 병무청 전문특기병으로 합격하여 육군 군악대로 복무 중이다. 두번째 뉴스 - 전 피겨스케이팅 선수 김연아가 근황을 공개했다. 김연아는 지난 6일 자신의 소셜미디어에 앰버서더(홍보대사)로 활동 중인 프랑스 럭셔리 브랜드와 함께 한 영상을 공유했다. 김연아는 지난달 24일 열린 서울 종로구 포시즌스 호텔에서 열린 '더블유 코리아(W KOREA) 제18회 유방암 인식 향상 캠페인' 자선 행사에 참석한 바 있다.
사진 속 김연아는 진주 단추가 달린 블랙 블라우스와 자수가 새겨진 조끼, 검은 치마를 입고 우아한 매력을 뽐냈다. 이를 본 누리꾼들은 "너무 예쁘네요", "여신이네요", "아름다워요" 등의 댓글을 달았다. 한편 김연아는 크로스오버 그룹 '포레스텔라' 멤버 고우림과 3년간 교제 끝에 지난해 10월 결혼식을 올렸다. 고우림은 지난달 20일 육군 현역으로 입대했으며, 육군 군악대로서 국방의 의무를 수행할 예정이다.
''',
"input":"",
"output":'''
두 뉴스의 내용을 비교해보면, 두 뉴스는 김연아의 근황 및 활동에 대한 내용이 다르며, 두 소식은 서로 관련이 없어 보입니다. 첫 번째 뉴스는 김연아의 남편 고우림의 군 복무에 대한 이야기이고, 두 번째 뉴스는 김연아의 최근 활동과 참석한 행사에 대한 내용입니다.
'''},
{"instruction":"두 기사를 읽고 내용의 유사도를 평가해줘",
"input": '''
article1 - 피겨 여왕 김연아의 남편, 고우림이 결혼 1년 만에 군 복무에 입대했다. 군 복무 이전의 고우림은 팀 포레스텔라의 멤버로 활동하며 뛰어난 실력과 훌륭한 인성으로 팬들의 사랑을 받았다. 결혼 후 1년간 행복한 신혼 생활을 즐겼지만, 최근 병무청 전문특기병으로 합격하여 육군 군악대로 복무 중이다. 두번째 기사 - 전 피겨스케이팅 선수 김연아가 근황을 공개했다. 김연아는 지난 6일 자신의 소셜미디어에 앰버서더(홍보대사)로 활동 중인 프랑스 럭셔리 브랜드와 함께 한 영상을 공유했다. 김연아는 지난달 24일 열린 서울 종로구 포시즌스 호텔에서 열린 '더블유 코리아(W KOREA) 제18회 유방암 인식 향상 캠페인' 자선 행사에 참석한 바 있다.
사진 속 김연아는 진주 단추가 달린 블랙 블라우스와 자수가 새겨진 조끼, 검은 치마를 입고 우아한 매력을 뽐냈다. 이를 본 누리꾼들은 "너무 예쁘네요", "여신이네요", "아름다워요" 등의 댓글을 달았다. 한편 김연아는 크로스오버 그룹 '포레스텔라' 멤버 고우림과 3년간 교제 끝에 지난해 10월 결혼식을 올렸다. 고우림은 지난달 20일 육군 현역으로 입대했으며, 육군 군악대로서 국방의 의무를 수행할 예정이다.
''',
"output":'''
두 뉴스의 내용을 비교해보면, 두 뉴스는 김연아의 근황 및 활동에 대한 내용이 다르며, 두 소식은 서로 관련이 없어 보입니다. 첫 번째 뉴스는 김연아의 남편 고우림의 군 복무에 대한 이야기이고, 두 번째 뉴스는 김연아의 최근 활동과 참석한 행사에 대한 내용입니다.
'''}
]

In [ ]:
features = ['instruction','input','output']

transformed_data = {
    'train': Dataset.from_dict({feature: [item[feature] for item in data] for feature in features})
}

dataset_dict = DatasetDict(transformed_data)

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 11
    })
})

In [ ]:
data = dataset_dict.map(lambda x: {'text':f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}"})

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 11
    })
})

#Koalpaca 모델과 토크나이저 설정

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/KoAlpaca-Polyglot-12.8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

...

model-00026-of-00028.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/517M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11
    })
})

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6607912960 || trainable%: 0.09917806181272702


#Koalpaca 파인튜닝

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.594300
20,1.719100
30,1.130800
40,0.475600
50,0.118700
60,0.052200
70,0.033700
80,0.020600
90,0.021000
100,0.018600


TrainOutput(global_step=500, training_loss=0.13602588960528375, metrics={'train_runtime': 3031.9866, 'train_samples_per_second': 0.33, 'train_steps_per_second': 0.165, 'total_flos': 2.92275679867392e+16, 'train_loss': 0.13602588960528375, 'epoch': 83.33})

In [ ]:
model.eval()
model.config.use_cache = True

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors = 'pt', return_token_type_ids = False))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1384: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1665: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[    6,     6,     6,  2438,    29,  1832,  4770,   272,    34,   202,
           202,     6,     6,     6,  3711,    29,  1832,  4770,   272, 12546]])

#대화를 주고받을 수 있게 하는 함수

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

#질문 테스트

In [ ]:
gen('''
건강하게 살기 위한 세가지 방법은?
''')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
건강하게 살기 위한 세가지 방법은?


### 답변: 1. 적절한 영양소 섭취 - 다양한 영양소를 적절한 비율로 섭취하여 건강한 몸을 만듭니다.
2. 꾸준한 운동 - 꾸준한 운동으로 건강한 몸을 유지합니다.
3. 긍정적인 생각과 웃음 - 긍정적인 생각과 웃음으로 건강을 키웁니다.

세가지 다 건강을 유지하는 간단한 방법이지만, 실제로 지키기 어려운 것 같아요. 그래도 최대한 지켜서 건강한 삶을 살아야겠죠? 부족한 답변이지만 도움이 되었으면 좋겠습니다.

### 답변:
건강하게 살기 위한 세 가지 방법은 다음과 같습니다.

1. 적절한 영양소 섭취 - 다양한 영양소를 적절한 비율로 섭취하여 건강한 몸을 만듭니다.

2. 꾸준한 운동 - 꾸준한 운동으로 건강한 몸을 유지합니다.

3. 긍정적인 생각과 웃음 - 긍정적인 생각과 웃음으로 건강을 키웁니다.

세 가지 방법은 건강을 유지하는 간단한 방법이지만, 실제로 지키기 어렵습니다. 그러나 최대한 지켜서 건강한 삶을 살아야 합니다. 부족한 답변이지만 도움이 되었으면 좋


In [ ]:
gen('''
뷔와 제니가 헤어졌다는 뉴스가 진짜야?
''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
뷔와 제니가 헤어졌다는 뉴스가 진짜야?


### 답변: 
사실입니다. 뷔와 제니가 같은 그룹의 멤버였다가 본연의 길을 가기로 했다는 말이다. 12월 20일 한 매체는 뷔가 가수 제니와의 연애에 마침표을 찍었다고 보도했다. 뷔의 소속사는 이와 관련해 어떤 입장도 내놓지 않았다. 반면 제니의 소속사는 두 사람의 결별 사실을 인정하며 "개인적인 일이기 때문에 신경 쓰지 않는다"라고 전했다. 두 사람과 동시에 연인 관계로 지낸 뷔와 제니는 지난 10월 말 태국의 한 리조트에서 휴가를 보냈다. 이 모습이 포착되면서 열애를 인정했고, 공개 연애를 시작했다. 하지만 결국 두 사람은 결별이라는 길을 걷게 됐다. 뷔과 제니과 연애에 마침표를 찍었다.

### 답변:사실입니다. 뷔와 제니가 같은 그룹의 멤버였다가 본연의 길을 가기로 했다는 소식이 전해졌으며, 뷔의 소속사는 이에 대한 어떤 입장도 내놓지 않았습니다. 제니의 소속사는 뷔와의 결별을 인정하며 "개인적인 일이기 때문에 신경 쓰지 않는다


In [ ]:
gen('''
두 기사를 읽고 내용의 유사도를 평가해줘.
첫번째 뉴스 - 피겨 여왕 김연아의 남편, 고우림이 결혼 1년 만에 군 복무에 입대했다. 군 복무 이전의 고우림은 팀 포레스텔라의 멤버로 활동하며 뛰어난 실력과 훌륭한 인성으로 팬들의 사랑을 받았다. 결혼 후 1년간 행복한 신혼 생활을 즐겼지만, 최근 병무청 전문특기병으로 합격하여 육군 군악대로 복무 중이다.
두번째 뉴스 - 전 피겨스케이팅 선수 김연아가 근황을 공개했다. 김연아는 지난 6일 자신의 소셜미디어에 앰버서더(홍보대사)로 활동 중인 프랑스 럭셔리 브랜드와 함께 한 영상을 공유했다. 김연아는 지난달 24일 열린 서울 종로구 포시즌스 호텔에서 열린 '더블유 코리아(W KOREA) 제18회 유방암 인식 향상 캠페인' 자선 행사에 참석한 바 있다.
사진 속 김연아는 진주 단추가 달린 블랙 블라우스와 자수가 새겨진 조끼, 검은 치마를 입고 우아한 매력을 뽐냈다. 이를 본 누리꾼들은 "너무 예쁘네요", "여신이네요", "아름다워요" 등의 댓글을 달았다. 한편 김연아는 크로스오버 그룹 '포레스텔라' 멤버 고우림과 3년간 교제 끝에 지난해 10월 결혼식을 올렸다. 고우림은 지난달 20일 육군 현역으로 입대했으며, 육군 군악대로서 국방의 의무를 수행할 예정이다.
''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
두 기사를 읽고 내용의 유사도를 평가해줘. 첫번째 뉴스 - 피겨 여왕 김연아의 남편, 고우림이 결혼 1년 만에 군 복무에 입대했다. 군 복무 이전의 고우림은 팀 포레스텔라의 멤버로 활동하며 뛰어난 실력과 훌륭한 인성으로 팬들의 사랑을 받았다. 결혼 후 1년간 행복한 신혼 생활을 즐겼지만, 최근 병무청 전문특기병으로 합격하여 육군 군악대로 복무 중이다. 두번째 뉴스 - 전 피겨스케이팅 선수 김연아가 근황을 공개했다. 김연아는 지난 6일 자신의 소셜미디어에 앰버서더(홍보대사)로 활동 중인 프랑스 럭셔리 브랜드와 함께 한 영상을 공유했다. 김연아는 지난달 24일 열린 서울 종로구 포시즌스 호텔에서 열린 '더블유 코리아(W KOREA) 제18회 유방암 인식 향상 캠페인' 자선 행사에 참석한 바 있다.
사진 속 김연아는 진주 단추가 달린 블랙 블라우스와 자수가 새겨진 조끼, 검은 치마를 입고 우아한 매력을 뽐냈다. 이를 본 누리꾼들은 "너무 예쁘네요", "여신이네요", "아름다워요" 등의 댓글을 달았다. 한편 김연아는 크로스오버 그룹 '포레스텔라' 멤버 고우림과 3년간 교제 끝에 지난해 10월 결혼식을 올렸다. 고우림은 지난달 20일 육군 현역으로 입대했으며, 육군 군악대로서 국방의 의무를 수행할 예정이다.


### 답변: 
두 뉴스의 내용을 비교해보면, 두 뉴스는 김연아의 근황 및 활동에 대한 내용이 다르며, 두 소식은 서로 관련이 없어 보입니다. 첫 번째 뉴스는 김연아의 남편 고우림의 군 복무에 대한 이야기이고, 두 번째 뉴스는 김연아의 최근 활동과 참석한 행사에 대한 내용입니다.

두 뉴스의 내용을 살펴보면, 첫 번째 뉴스는 팀 포레스텔라의 멤버 고우림이 군 복무를 위해 입대했다는 보도이며, 두 번째 뉴스는 김연아의 최근 활동과 참석한 행사에 대한 내용이 담겨 있습니다.

두 뉴스 간에는 서로 관련이 없으며, 두 뉴스는 독립적인 내용입니다. 김연아의 남편 고우림의 군 복무와 이에 대한 소식이 포함된 첫 번째 뉴스와, 김연아의 최근 활동과 참석한 

In [ ]:
gen('''
빅뱅이 해체했다는 뉴스가 진짜야?
''')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1665: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 
빅뱅이 해체했다는 뉴스가 진짜야?


### 답변: (두 가지 뉴스를 바탕으로 종합해서 판단해보면) 사실인 것으로 보입니다. 첫 번째 뉴스는 '빅뱅 멤버 4명이서만 활동 재개해'이며, 두 번째 뉴스는 '빅뱅 결국 해체... 지수·제뉴 음성에도 YG 측 경영 수업 축소'입니다. 두 뉴스를 살펴보면 멤버들은 변함이 없으며, 그룹의 완전한 해체은 아닌 것으로 보입니다. 그러나, 멤버들의 계약 만료, 탈퇴, 또는 배우나 솔로 활동으로 인해 그룹이 해체된 것으로 추정됩니다.

### 답변:진실로 보입니다. 첫 번째 뉴스는 '빅뱅 멤버 4명이서만 활동 재개해'이며, 두 번째 뉴스는 '빅뱅 결국 해체... 지수·제뉴 음성에도 YG 측 경영 수업 축소'입니다. 두 뉴스를 살펴보면 멤버들은 변함이 없으며, 그룹의 완전한 해체는 아닌 것으로 보입니다. 그러나, 멤버들의 계약 만료, 탈퇴, 또는 배우나 솔로 활동으로 인해 그룹이 해체된 것으로 추정됩니다. 이는 2018년 5월 12일에 발행된 뉴스 기사에 나와있습니다. 

또한, 두 뉴스 기사를 살펴보면 YG엔터테인


In [ ]:
gen('''

''')